In [210]:
import numpy as np
import pandas as pd

In [211]:
!pip3 install pyvi

In [212]:
from pyvi import ViPosTagger, ViTokenizer

In [213]:
data_train = pd.read_csv('./data_train.csv')

In [214]:
data_train = data_train.sample(frac = 1).reset_index(drop = True)
X = data_train[['Review']]
y = data_train['Label'].fillna(0.)
print(X.shape[0])

4140


In [215]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)

In [216]:
# Loại bỏ kí tự đặc biệt và chuyển thành chữ cái viết thường
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P)_ '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text


In [217]:
def clean(data):
  for i in data.index:
    data.loc[i, "Review"] = ViTokenizer.tokenize(preprocessor(data.loc[i, "Review"]))
  return data

In [218]:
def splitSentence(text):
  list_of_sentences = text.split('.')
  return list_of_sentences
print(splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you"))
a,b, c = splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you")
print(a)
print(b)
print(c)

['dùng khá ổn', ' Cực kì ủng hộ', 'Ilove you']
dùng khá ổn
 Cực kì ủng hộ
Ilove you


In [219]:
splitter = X.shape[0]*0.7
X_train = X.loc[:splitter]
y_train = y.loc[:splitter]
X_test = X.loc[splitter:]
y_test = y.loc[splitter:]

In [220]:
with open('./stopwords_vn_dash.txt') as f:
    stopwords_vn = f.read().splitlines()

In [221]:
X_train = clean(X_train)
#print(X_train.loc[3444])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [222]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
X_train_bow = bow.fit_transform(X_train["Review"])
print(X_train_bow.shape)
print(bow.get_feature_names())

(2899, 14145)
['abc', 'about', 'about the', 'ac', 'ac fody', 'ace', 'ad', 'adayroi', 'aen', 'aeon', 'aeon mal', 'again', 'ahihi', 'ai_ngờ', 'ai_đời', 'al', 'al fresco', 'al the', 'alacarte', 'almaz', 'alo', 'always', 'am', 'amount', 'amount of', 'an', 'an hành_tây', 'an thi', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and not', 'and the', 'and there', 'and they', 'and văn_bản', 'anh_chị', 'anh_chị chủ', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị đi', 'anh_chị_em', 'anh_chị_em công_sở', 'anh_em', 'anh_em quán', 'anh_vũ', 'any', 'ap', 'are', 'are high', 'are realy', 'aroi', 'as', 'at', 'authentic', 'avalon', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ ngon', 'ba_chỉ nướng', 'ba_chỉ nấm', 'ba_chỉ quán', 'back', 'bacon', 'bad', 'baguete', 'bakery', 'bal', 'bami', 'ban', 'ban_công', 'ban_công không', 'ban_ngày', 'ban_ngày không', 'ban_đầu', 'ban_đầu quán', 'banana', 'banh', 'bao', 'bao gọi', 'bao không', 'bao kim_sa', 'bao quán', 'bao trọn', 'bao đi', 'bao_bì', 'bao_cấp', 'bao_gồm', 'bao_gồm 

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
# Feed the tf-idf Vectorizer with twits using fit_transform()
X_train_tfidf = tfidf.fit_transform(X_train["Review"])

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(X_train_tfidf.shape)

['abc', 'about', 'about the', 'ac', 'ac fody', 'ace', 'ad', 'adayroi', 'aen', 'aeon', 'aeon mal', 'again', 'ahihi', 'ai_ngờ', 'ai_đời', 'al', 'al fresco', 'al the', 'alacarte', 'almaz', 'alo', 'always', 'am', 'amount', 'amount of', 'an', 'an hành_tây', 'an thi', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and not', 'and the', 'and there', 'and they', 'and văn_bản', 'anh_chị', 'anh_chị chủ', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị đi', 'anh_chị_em', 'anh_chị_em công_sở', 'anh_em', 'anh_em quán', 'anh_vũ', 'any', 'ap', 'are', 'are high', 'are realy', 'aroi', 'as', 'at', 'authentic', 'avalon', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ ngon', 'ba_chỉ nướng', 'ba_chỉ nấm', 'ba_chỉ quán', 'back', 'bacon', 'bad', 'baguete', 'bakery', 'bal', 'bami', 'ban', 'ban_công', 'ban_công không', 'ban_ngày', 'ban_ngày không', 'ban_đầu', 'ban_đầu quán', 'banana', 'banh', 'bao', 'bao gọi', 'bao không', 'bao kim_sa', 'bao quán', 'bao trọn', 'bao đi', 'bao_bì', 'bao_cấp', 'bao_gồm', 'bao_gồm đậu', 'bao_gồm

In [224]:
clean(X_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Review
2898,đã vào ngọc_thạch quán nhiều lần và lần nào cũ...
2899,hàng bánh địa_điểm hơi xấu nhưng không_gian qu...
2900,quá này phục_vụ quá đến ngồi đợi phút không có...
2901,mình đi khá đông nhưng ít nhân_viên gọi mãi ch...
2902,nhân_viên phục_vụ hơi lạnh_lùng chắc tại quán ...
...,...
4135,lâu mới đi ăn lại mà cảm_giác không ngon như n...
4136,nghe chago đã lâu hôm nọ mình đến gọi cốc trà ...
4137,giá_cả phải_chăng món ăn thì quá chi là hấp_dẫn
4138,mặc_dù quán nằm trong ngõ nhưng lại không khó ...


In [225]:
def splitTest(X_test):
  X_test_split = pd.DataFrame(columns = ["Sentence","Origin"])
  for i in X_test.index:
    los = splitSentence(X_test.loc[i,"Review"])
    for j in range (len(los)):
      new_row = {"Sentence": los[j], "Origin": i}
      X_test_split = X_test_split.append(new_row, ignore_index= True)
  return X_test_split  


In [226]:
def concatPred(y_pred, X_test_split):
  #print(len(y_pred))
  X_test_split["Predict"] = y_pred
  X_test_split
  X_test_split = X_test_split.groupby("Origin").sum()
  y_pred = X_test_split["Predict"].tolist()
  for i in range(len(y_pred)):
    if y_pred[i] > 0:
      y_pred[i] = 1.
    elif y_pred[i] <0: 
      y_pred[i] = -1.
    else:
      y_pred[i] = 0.
  #print(len(y_pred))
  return y_pred

In [227]:
X_test_split = splitTest(X_test)

In [228]:
X_test_bow = bow.transform(X_test_split["Sentence"])
X_test_tfidf = tfidf.transform(X_test_split["Sentence"])

In [229]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

In [230]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [231]:
nbc.fit(X_train_bow.toarray(), y_train)
y_pred = nbc.predict(X_test_bow.toarray())
y_pred = concatPred(y_pred, X_test_split)

1242
1242


In [232]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.78341384863124
0.5613294982422499


In [233]:
nbc.fit(X_train_tfidf.toarray(), y_train)
y_pred = nbc.predict(X_test_tfidf.toarray())
y_pred = concatPred(y_pred, X_test_split)

1242
1242


In [234]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7874396135265701
0.5538421991848597


In [235]:
from sklearn.linear_model import LogisticRegression


In [236]:
lor = LogisticRegression()
lor.fit(X_train_bow.toarray(), y_train)
y_pred = lor.predict(X_test_bow.toarray())
concatPred(y_pred, X_test_split)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

1242
1242
0.7624798711755234
0.5830800670858597


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [254]:
lor.fit(X_train_tfidf.toarray(), y_train)
y_pred = lor.predict(X_test_tfidf.toarray())
y_pred = concatPred(y_pred, X_test_split)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))
print(classification_report(y_test, y_pred))

1242
1242
0.749597423510467
0.5787546476139145
              precision    recall  f1-score   support

        -1.0       0.75      0.76      0.76       496
         0.0       0.29      0.10      0.15       139
         1.0       0.78      0.89      0.83       607

    accuracy                           0.75      1242
   macro avg       0.61      0.58      0.58      1242
weighted avg       0.71      0.75      0.72      1242



In [238]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth = None, random_state = 0)

In [239]:
rfc.fit(X_train_bow.toarray(), y_train)
y_pred = rfc.predict(X_test_bow.toarray())
concatPred(y_pred, X_test_split)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

1242
1242
0.7576489533011272
0.5508908969702745


In [240]:
rfc.fit(X_train_tfidf.toarray(), y_train)
y_pred = rfc.predict(X_test_tfidf.toarray())
concatPred(y_pred, X_test_split)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

1242
1242
0.7697262479871175
0.5634932565600981


In [241]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

In [242]:
mlp.fit(X_train_bow, y_train)
y_pred = mlp.predict(X_test_bow)
concatPred(y_pred, X_test_split)
print(classification_report(y_test, y_pred))

1242
1242
              precision    recall  f1-score   support

        -1.0       0.73      0.74      0.73       496
         0.0       0.16      0.12      0.14       139
         1.0       0.80      0.83      0.82       607

    accuracy                           0.71      1242
   macro avg       0.56      0.56      0.56      1242
weighted avg       0.70      0.71      0.71      1242



In [243]:
mlp.fit(X_train_tfidf, y_train)
y_pred = mlp.predict(X_test_tfidf)
concatPred(y_pred, X_test_split)
print(classification_report(y_test, y_pred))

1242
1242
              precision    recall  f1-score   support

        -1.0       0.76      0.78      0.77       496
         0.0       0.30      0.13      0.18       139
         1.0       0.81      0.89      0.85       607

    accuracy                           0.76      1242
   macro avg       0.62      0.60      0.60      1242
weighted avg       0.73      0.76      0.74      1242



In [244]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler(with_mean= False)
X_train_bow = ss.fit_transform(X_train_bow)
X_test_bow = ss.transform(X_test_bow)
X_train_tfidf = ss.fit_transform(X_train_tfidf)
X_test_tfidf  = ss.transform(X_test_tfidf)

In [245]:
from sklearn.svm import SVC
clf_svc = SVC(C = 1.)



In [246]:
clf_svc.fit(X_train_bow, y_train)
y_pred = clf_svc.predict(X_test_bow)
concatPred(y_pred, X_test_split)
print(classification_report(y_test, y_pred))

1242
1242
              precision    recall  f1-score   support

        -1.0       0.75      0.46      0.57       496
         0.0       0.00      0.00      0.00       139
         1.0       0.60      0.92      0.72       607

    accuracy                           0.64      1242
   macro avg       0.45      0.46      0.43      1242
weighted avg       0.59      0.64      0.58      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [247]:
clf_svc.fit(X_train_tfidf, y_train)
y_pred = clf_svc.predict(X_test_tfidf)
concatPred(y_pred, X_test_split)
print(classification_report(y_test, y_pred))

1242
1242
              precision    recall  f1-score   support

        -1.0       0.68      0.73      0.70       496
         0.0       0.00      0.00      0.00       139
         1.0       0.71      0.82      0.76       607

    accuracy                           0.69      1242
   macro avg       0.46      0.52      0.49      1242
weighted avg       0.62      0.69      0.65      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [248]:
import pickle

In [253]:
file_name = "lor_model.sav"
file_name_tfidf = "tfidf_transform.sav"
pickle.dump(lor, open(file_name, 'wb'))
pickle.dump(tfidf, open(file_name_tfidf, 'wb'))

In [252]:
loaded_model = pickle.load(open(file_name, 'rb'))
print(loaded_model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
